In [1]:
import os
import findspark
findspark.init(os.environ['SPARK_HOME'])

from pyspark.sql import SparkSession

# Inicializando a sessão do Spark
spark = SparkSession.builder.appName("Exemplo").getOrCreate()

## Text Cleaning and NLP
---

- **Remoção de Stopwords:** Eliminar palavras comuns que não contribuem significativamente para a análise.


In [2]:
import nltk
from nltk.corpus import stopwords

from pyspark.ml.feature import StopWordsRemover, RegexTokenizer
from pyspark.sql.functions import col

In [3]:
# Baixar a lista de stopwords em português do NLTK
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\aview\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
# Obter a lista de stopwords em português
stopwords_portugues = set(stopwords.words('portuguese'))

In [5]:
# Total de stopwords em portugues: 
print(f"Total de stopwords em portugues: {len(stopwords_portugues)}")

# Exibir as primeiras 10 stopwords
print("Lista de Stopwords em Português apenas 10:")
print(list(stopwords_portugues)[:10])

Total de stopwords em portugues: 207
Lista de Stopwords em Português apenas 10:
['fora', 'seu', 'com', 'teve', 'como', 'somos', 'houverá', 'mas', 'estávamos', 'houverei']


In [8]:
# Criando um DataFrame de exemplo com uma coluna de texto
data = [("Documento1", "O processamento de texto é interessante"),
        ("Documento2", "Outro exemplo de tokenização")]

columns = ["Documento", "Texto"]

df = spark.createDataFrame(data, columns)

# Exibindo o DataFrame original
print("DataFrame Original:")
df.show(truncate=False)

DataFrame Original:
+----------+---------------------------------------+
|Documento |Texto                                  |
+----------+---------------------------------------+
|Documento1|O processamento de texto é interessante|
|Documento2|Outro exemplo de tokenização           |
+----------+---------------------------------------+



In [9]:
# Configurando o tokenizador
tokenizer = RegexTokenizer(inputCol="Texto", outputCol="Tokens", pattern="\\W")

# Aplicando a tokenização
df_tokenizado = tokenizer.transform(df)

# Exibindo o DataFrame resultante
print("\nDataFrame Após Tokenização:")
df_tokenizado.select("Documento", "Tokens").show(truncate=False)


DataFrame Após Tokenização:
+----------+-------------------------------------------+
|Documento |Tokens                                     |
+----------+-------------------------------------------+
|Documento1|[o, processamento, de, texto, interessante]|
|Documento2|[outro, exemplo, de, tokeniza, o]          |
+----------+-------------------------------------------+



In [10]:
# Configurando o remover de stopwords
stopwords_remover = StopWordsRemover(inputCol="Tokens", outputCol="TextoSemStopwords", stopWords=list(stopwords_portugues))

In [11]:
# Aplicando a remoção de stopwords (note que você deve ter a coluna "Tokens" do exemplo anterior)
df_sem_stopwords = stopwords_remover.transform(df_tokenizado)

In [12]:
# Exibindo o DataFrame resultante
print("\nDataFrame Após Remoção de Stopwords:")
df_sem_stopwords.select("Documento", "TextoSemStopwords").show(truncate=False)


DataFrame Após Remoção de Stopwords:
+----------+------------------------------------+
|Documento |TextoSemStopwords                   |
+----------+------------------------------------+
|Documento1|[processamento, texto, interessante]|
|Documento2|[outro, exemplo, tokeniza]          |
+----------+------------------------------------+



In [36]:
spark.stop()